<a href="https://colab.research.google.com/github/Harini766/Missing-Word-Detection-Using-BERT-Model/blob/main/Missing_Word_Detection_Using_BERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import necessary libraries
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [2]:
# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [5]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [6]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"{mask} like cricket and {mask} like football too"
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['[MASK]', 'like', 'cricket', 'and', '[MASK]', 'like', 'football', 'too']


In [7]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[ 101,  103, 1176, 5428, 1105,  103, 1176, 1709, 1315,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [8]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3620,  -7.2552,  -7.3831,  ...,  -6.0268,  -5.9224,  -6.3655],
         [ -5.0954,  -5.3102,  -5.1809,  ...,  -4.2193,  -3.7114,  -4.7352],
         [-10.9880, -10.7406, -10.9404,  ...,  -7.7151,  -8.3863,  -7.9467],
         ...,
         [ -6.5436,  -6.6154,  -6.9540,  ...,  -3.6041,  -5.4135,  -6.5668],
         [ -9.0178,  -8.7913,  -9.0128,  ...,  -6.1427,  -7.2575,  -7.7417],
         [-12.3347, -12.8596, -12.2670,  ..., -10.0364, -10.8458, -10.5040]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [9]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(10, 28996)

In [10]:
len(tokens)

8

In [11]:
tokens.index(mask)

0

In [12]:
# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-5.0954075 -5.3101826 -5.180928  ... -4.2193236 -3.711428  -4.735197 ]


In [13]:
# Calculate the confidence scores for each possible token using softmax
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[6.3180643e-09 5.0969269e-09 5.8001928e-09 ... 1.5172677e-08 2.5213831e-08
 9.0577723e-09]


In [14]:
# Calculate the confidence scores for each possible token using softmax
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[6.3180643e-09 5.0969269e-09 5.8001928e-09 ... 1.5172677e-08 2.5213831e-08
 9.0577723e-09]


In [15]:
confidence_scores.sum()

1.0

In [16]:
# Print the top 5 predicted tokens and their confidence scores
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I like cricket and I like football too 0.75715196
They like cricket and They like football too 0.075008534
You like cricket and You like football too 0.05915661
We like cricket and We like football too 0.031155633
People like cricket and People like football too 0.0050241756
